# 문제점 : 분류 문제에서 잔차를 구한다는 것은 어떠한 의미인가? 

예시로 이진 분류에서 y값이 -1 / +1 둘 중 하나를 띈다고 가정할 때, 
y 값의 list가 [1,1,-1-1] 이나, $f_1(x)$ 가 [1, -1, 1, -1] 로 예측했다고 하자. 
그럼 잔차 (y - f(x)) = [0 , 2, -2, 0] 이 된다. 

이때 -2, 0, 2 는 모두 기존 분류에서 없던 값들이다. 그렇다면 잔차를 다시 추정하는 $f_2(x)$ 는 $f_1(x)$와 달리 3분류를 진행하는 것인가? 

$f_n(x)$ 함수는 계속해서 Class의 개수가 늘어날 텐데 이렇게 계산하는 것이 맞나? 


# Ensemble Learning - GMB 

### 수도 알고리즘은 강의 자료 참고 

1. Initialize $f_0(x)$ = $argmin_\gamma \sum_{i=1}^N L(y_i, \gamma)$ 
2. For m = 1 to M 
- for i = 1, ... N compute 
> $g_{im} = [\frac{\partial L(y_i, f(x_i))}{\partial f(x_i)}]_{f(x_i) = f_{m-1}(x_i)}$ 
- Fit a regression tree to the targets $g_{im}$ giving terminal regions $R_{jm}, j=1, ... , J_m$ 
- For j=1, ..., $J_m$ compute 
> $\gamma_jm$ = $argmin_\gamma \sum_{x_i \in R_{jm}} L(y_i, f_{m-1}(x_i) + \gamma)$
- update $f_m(x) = f_{m-1}(x) + \sum_{j=1}^{J_m} \gamma_{jm} I(x \in R_{jm})$ 
3. Output $\hat f(x) = f_M(x)$ 



**구현해야 하는 것**
- $f_0(x)$ = $argmin_\gamma \sum_{i=1}^N L(y_i, \gamma)$  : 임의의 Stump tree 진행. 
- $\gamma_jm$ = $argmin_\gamma \sum_{x_i \in R_{jm}} L(y_i, f_{m-1}(x_i) + \gamma)$ : y- $f_1(x)$ 로 Dataset의 값을 변경시키고, 이를 예측하는 Stump tree 적용
* 원래는 함수를 더해가는 게 맞으나, 함수를 더하는 과정을 어떻게 구현할지 감이 안옴. 이에 데이터를 변경시키는 쪽으로 구현하겠음. 

- f(x) : 약한 학습기. Stump tree 적용 
- loss 함수 : OLS 적용
- Aggregation 함수 : Majority 함수 채택 

**필요한 것** 
- Data : Stump Tree를 적용할 것으로 y값이 1, -1로 구분될 것 
- M : 트리 개수
- Aggregating 방식 




In [1]:
import numpy as np
import pandas as pd
import random as rand

from sklearn.datasets import load_iris
X = load_iris()['data'][:100]

# y의 값을 +1, -1 둘 중 하나로 변경 
y = load_iris()["target"][:100]
y[:50] = -1
y= y.reshape(-1,1)
S = np.concatenate((X,y), axis=1)

import matplotlib.pyplot as plt
import scipy as sc
from scipy.stats import norm
from sys import maxsize

In [ ]:
class GBM() : 
    def __init__(self, )

In [ ]:
# stump_tree 함수 구현 
def stump_tree(data) : 
    chose_var = data[np.random.choice(range(len(data)))]
    chose_att = np.random.choice(range(np.shape(data)[1]-1))
    crit = chose_var[chose_att]
    
    left = [] 
    right = [] 
    result = np.zeros(len(data))
    for index in range(len(data)) : 
        if data[index][chose_att] > crit : right.append(index)
        else : left.append(index)
    
    right_result = [1 if data[right][i,-1] == 1 else 0 for i in range(len(right)) ] 
    left_result = [1 if data[left][i,-1] == -1 else 0 for i in range(len(left)) ]
    if np.sum(right_result) + np.sum(left_result) > len(data)/2 : 
        result[right] = 1 
        result[left] = -1 
        direction = "right" 
    else : 
        result[right] = -1 
        result[left] = 1 
        direction = "left"
    return result, chose_att, crit, direction 

def cal_stump_tree(vector, chose_att, crit, direction) :
    if vector[chose_att] > crit :
        if direction == "right":  return 1
        else : return -1 
        
    else : 
        if direction == "right" : return -1 
        else : return 1 

In [ ]:
def adaboost(T,data, new_vector) : 
    D_list = [] 
    D = np.ones(len(data)) / len(data) 
    D_list.append(D)
    
    alpha_list = []
    h_list = [] 
    
    for t in range(T) :
        Z = 0 
        
        # 아래처럼 Random Choice로 샘플을 골라내는 게 맞을까? 
        new_index = np.random.choice(range(len(data)), len(data), p=D)
        new_data = data[new_index]
        epsil = epsilon(new_data)
        
        h, chose_att, crit, direction = stump_tree(new_data)
        h_list.append([chose_att, crit, direction])
        a = alpha(epsil)
        alpha_list.append(a)
        new_D = [] 
        for i in range(len(new_data)) : 
            new_D_value =  D[i]* np.exp(-a * new_data[i][-1] * h[i])
            Z += new_D_value
            new_D.append(new_D_value)
        
        D = np.array(new_D)/Z
    
    result = 0 
    for t in range(T):
        chose_att, crit, direction = h_list[t]
        result += alpha_list[t] * cal_stump_tree(new_vector, chose_att, crit, direction)
    
    if result > 0 : return 1
    else : return -1 


In [ ]:
#엔트로피 설정

from math import log 
from collections import defaultdict

def entropy(dataset) : 
    datasize = len(dataset) 
    
    label_count = defaultdict(int)
    entropy = 0.0 
    
    for data_line in dataset : 
        cur_label = data_line[-1]
        label_count[cur_label] += 1 
        
    for key in label_count : 
        prop = float(label_count[key]) / datasize 
        entropy -= prop*log(prop,2) 
    return entropy

# Dv, Ent(Dv) 계산하기 위한 토대. 특정 속성값을 제외한 나머지 값들을 결과로 반환 
def pick(dataset, index, value) : 
    result = [] 
    for dataline in dataset : 
        dataline = list(dataline)
        if dataline[index] == value : 
            temp_vec = dataline[:index]
            temp_vec.extend(dataline[index+1:])
            result.append(temp_vec)
    return result 

# 최고의 선별 속성을 결정하는 함수 
def best_feat(dataset): 
    base_entropy = entropy(dataset)
    best_infogain = 0.0 
    index = 0 
    num_feat = len(dataset[0]) -1 
    for i in range(num_feat) : 
        feat_value_set = set([value[i] for value in dataset])
        new_entropy = 0.0 
        
        # 정보 이득 계산하기 
        for element in feat_value_set : 
            sub_dataset = pick(dataset, i, element) 
            prop = len(sub_dataset) / float(len(dataset))
            new_entropy += prop*entropy(sub_dataset)
            
        info_gain = base_entropy - new_entropy 
        
        if best_infogain < info_gain : 
            best_infogain = info_gain 
            index = i 
    return index 